In [1]:
from collections import namedtuple
import scipy.stats as sps
import statsmodels.stats.api as sms
from tqdm.notebook import tqdm as tqdm_notebook # tqdm – библиотека для визуализации прогресса в цикле
from collections import defaultdict
from statsmodels.stats.proportion import proportion_confint
import numpy as np
import itertools
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.5, palette='Set2')
ExperimentComparisonResults = namedtuple('ExperimentComparisonResults', 
                                        ['pvalue', 'effect', 'ci_length', 'left_bound', 'right_bound'])

# Bootstrap

In [2]:
def bootstrap(control, test, test_type='absolute'):
    # Функция от средних, которую надо посчитать на каждой выборке
    absolute_func = lambda C, T: T - C
    relative_func = lambda C, T: T / C - 1
    
    boot_func = absolute_func if test_type == 'absolute' else relative_func
    stat_sample = []
    
    batch_sz = 100
    
    #В теории boot_samples_size стоить брать не меньше размера выборки. Но на практике можно и меньше.
    boot_samples_size = len(control)
    for i in range(0, boot_samples_size, batch_sz):
        N_c = len(control)
        N_t = len(test)
        # выбираем N_c элементов с повторением из текущей выборки. 
        # И чтобы ускорить этот процесс, делаем это сразу batch_sz раз
        # Вместо одной выборки мы получим batch_sz выборок
        control_sample = np.random.choice(control, size=(len(control), batch_sz), replace=True)
        test_sample    = np.random.choice(test, size=(len(test), batch_sz), replace=True)

        C = np.mean(control_sample, axis=0)
        T = np.mean(test_sample, axis=0)
        assert len(T) == batch_sz
        
        # добавляем в массив посчитанных ранее статистик batch_sz новых значений
        # X в статье – это boot_func(control_sample_mean, test_sample_mean)
        stat_sample += list(boot_func(C, T))

    stat_sample = np.array(stat_sample)
    # считаем истинный эффект
    effect = boot_func(np.mean(control), np.mean(test))
    left_bound, right_bound = np.quantile(stat_sample, [0.025, 0.975])
    
    ci_length = (right_bound - left_bound)
    # pvalue - процент статистик, которые лежат левее или правее 0.
    pvalue = 2 * min(np.mean(stat_sample > 0), np.mean(stat_sample < 0))
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)


AB-тест

In [3]:
# 3. Заводим счетчик
bad_cnt = 0

# 4. Цикл проверки
N = 10000
for i in tqdm_notebook(range(N)):
    # 4.a. Тестирую AB - тест
    control = sps.expon(scale=1000).rvs(2000)
    test = sps.expon(scale=1000).rvs(2300)
    test *= 1.1

    # 4.b. Запускаю критерий
    _, _, _, left_bound, right_bound = bootstrap(control, test, 'relative')
    
    # 4.c. Проверяю, лежит ли истинная разница средних в доверительном интервале
    if left_bound > 0.1 or right_bound < 0.1:
        bad_cnt += 1

# 5. Строю доверительный интервал для конверсии ошибок у критерия.
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, alpha=0.05, method='wilson')
# Результат
print(f"Реальный уровень значимости: {round(bad_cnt / N, 4)};"
      f" доверительный интервал: [{round(left_real_level, 4)}, {round(right_real_level, 4)}]")

  0%|          | 0/10000 [00:00<?, ?it/s]

Реальный уровень значимости: 0.0525; доверительный интервал: [0.0483, 0.057]


# Пост-нормировка

In [6]:
def post_normed_bootstrap(control, test, control_before, test_before, test_type='absolute'):
    # Функция от средних, которую надо посчитать на каждой выборке
    absolute_func = lambda C, T, C_b, T_b: T - (T_b / C_b) * C
    relative_func = lambda C, T, C_b, T_b: (T / C) / (T_b / C_b) - 1
    
    boot_func = absolute_func if test_type == 'absolute' else relative_func
    stat_sample = []
    
    batch_sz = 100
    
    #В теории boot_samples_size стоить брать не меньше размера выборки. Но на практике можно и меньше.
    boot_samples_size = len(control)
    for i in range(0, boot_samples_size, batch_sz):
        N_c = len(control)
        N_t = len(test)
        # Надо помнить, что мы семплируем именно юзеров
        # Поэтому, если мы взяли n раз i элемент в выборке control
        # То надо столько же раз взять i элемент в выборке control_before
        # Поэтому будем семплировать индексы
        control_indices = np.arange(N_c)
        test_indices = np.arange(N_t)
        control_indices_sample = np.random.choice(control_indices, size=(len(control), batch_sz), replace=True)
        test_indices_sample    = np.random.choice(test_indices, size=(len(test), batch_sz), replace=True)

        C   = np.mean(control[control_indices_sample], axis=0)
        T   = np.mean(test[test_indices_sample], axis=0)
        C_b = np.mean(control_before[control_indices_sample], axis=0)
        T_b = np.mean(test_before[test_indices_sample], axis=0)
        assert len(T) == batch_sz
        stat_sample += list(boot_func(C, T, C_b, T_b))

    stat_sample = np.array(stat_sample)
    # считаем истинный эффект
    effect = boot_func(np.mean(control), np.mean(test), np.mean(control_before), np.mean(test_before))
    left_bound, right_bound = np.quantile(stat_sample, [0.025, 0.975])
    
    ci_length = (right_bound - left_bound)
    # pvalue - процент статистик, которые лежат левее или правее 0.
    pvalue = 2 * min(np.mean(stat_sample > 0), np.mean(stat_sample < 0))
    return ExperimentComparisonResults(pvalue, effect, ci_length, left_bound, right_bound)

In [7]:
# 3. Заводим счетчик
bad_cnt = 0

# 4. Цикл проверки
N = 30000
cis = []
for i in tqdm_notebook(range(N)):
    # 4.a. Тестирую AB - тест
    control_before = sps.expon(scale=1000).rvs(1000)
    control = control_before + sps.norm(loc=0, scale=100).rvs(1000)

    test_before = sps.expon(scale=1000).rvs(1000)
    test = test_before + sps.norm(loc=0, scale=100).rvs(1000)
    test *= 1.1

    # 4.b. Запускаю критерий
    _, _, ci, left_bound, right_bound = post_normed_bootstrap(control, test, control_before, test_before, 'relative')
    cis.append(ci)
    # 4.c. Проверяю, лежит ли истинная разница средних в доверительном интервале
    if left_bound > 0.1 or right_bound < 0.1:
        bad_cnt += 1

# 5. Строю доверительный интервал для конверсии ошибок у критерия.
left_real_level, right_real_level = proportion_confint(count = bad_cnt, nobs = N, alpha=0.05, method='wilson')
# Результат
print(f"Реальный уровень значимости: {round(bad_cnt / N, 4)};"
      f" доверительный интервал: [{round(left_real_level, 4)}, {round(right_real_level, 4)}]")

  0%|          | 0/30000 [00:00<?, ?it/s]

Реальный уровень значимости: 0.0513; доверительный интервал: [0.0489, 0.0539]
